In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import json
import cv2
import matplotlib.pyplot as plt

In [ ]:
ratios_file = open("ratios.json", "r")
#print(len(ratios_file.read()))
ratios = json.load(ratios_file)
ratios_file.close()

In [ ]:
instruments = ["BTC"]
periods = list(map(lambda x: int(x), os.listdir("images_reduced/" + instruments[0] + "/")))
period = np.max(periods)
folder = "images_reduced/" + instruments[0] + "/" 
folder

In [ ]:
periods

In [ ]:
def prep_img(img_name):
    img = np.array(cv2.imread(img_name))
    if img.shape == ():
        raise Exception("none type of img1 " + img_name)
    img = np.array(img[:,:,0] > 240, dtype = np.float16)
    return img.reshape(img.shape[0], img.shape[1], 1)
def prep_BB(img_name):
    f = np.concatenate
    img = np.array(cv2.imread(img_name))
    if img.shape == ():
        raise Exception("none type of img2 " + img_name)
    out_shape = (img.shape[0], img.shape[1], 1)
    BB_low = np.logical_not(np.logical_and(img[:,:,0]<240,img[:,:,2]<240)).reshape(out_shape)
    BB_mid = np.logical_not(np.logical_and(img[:,:,0]>240,img[:,:,2]<240)).reshape(out_shape)
    BB_high1 = np.logical_and(img[:,:,0]<180,img[:,:,0]>50)
    BB_high2 = np.logical_and(img[:,:,1]<180,img[:,:,2]>250)
    BB_high = np.logical_and(BB_high1,BB_high2).reshape(out_shape)
    BB_high = np.logical_not(BB_high)
    return f((f((BB_low,BB_mid),axis=2), BB_high),axis=2).astype(np.float16)

In [ ]:
X = []
y = []
dates = np.sort(list(map(lambda x:x.split(";")[0], filter(lambda x: "close" in x, os.listdir(folder+ str(period) + "/")))))

for date in dates:
    cont = False
    img = prep_img(folder + str(period) + "/" + date + ";close.png")
    channels_in = img
    cat = "images_reduced/"
    try:
        for instrument in instruments:
            cat += instrument + "/" 

            for per in periods: 
                cata = cat + str(per) + "/"
                img = prep_BB(cata + date + ";" + "BB" + ".png")
                channels_in = np.concatenate((channels_in, img), axis = 2)
                for ind in ['ma', 'ADX0','ATR0', 'ARO0', 'WLR0']:
                    img = prep_img(cata + date + ";" + ind + ".png")
                    channels_in = np.concatenate((channels_in, img), axis = 2)
    except Exception as e:
        print(e)
        cont = True

    if not date in ratios.keys() or cont:
        continue
    y.append(ratios[date])
    X.append(channels_in)
X = np.array(X, dtype=np.float16)
y = np.array(y, dtype=np.float16)

In [ ]:
X.shape, y.shape

In [ ]:
for i in range(0, X.shape[0], X.shape[0]//5):
    for j in range(X.shape[-1]):
        plt.imshow(X[i,:,:,j])
        plt.show()

In [ ]:
inputs = keras.Input(shape=(X.shape[1], X.shape[2], X.shape[-1]),dtype=tf.float16)

xl = layers.Conv2D(64, 2, activation="relu")(inputs)
xl = layers.Conv2D(64, 2, activation="relu")(xl)
xl = layers.Conv2D(64, 2, activation="relu")(xl)
xl = layers.MaxPooling2D(2)(xl)
xl = layers.Conv2D(32, 2, activation="relu")(xl)
xl = layers.Conv2D(32, 2, activation="relu")(xl)
xl = layers.Conv2D(32, 2, activation="relu")(xl)
xl = layers.MaxPooling2D(2)(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.MaxPooling2D(2)(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.MaxPooling2D(2)(xl)
xl = layers.Conv2D(16, 2, activation="relu")(xl)
xl = layers.Flatten()(xl)
xl = dense = layers.Dense(8, activation="relu")(xl)
xl = dense = layers.Dense(4, activation="relu")(xl)
outputs = dense = layers.Dense(2)(xl)
model = keras.Model(inputs=inputs, outputs=outputs, name="algotrading")

In [ ]:
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.RMSprop())
model.summary()

In [ ]:
X = tf.constant(X, dtype = tf.float16)
y = tf.constant(y, dtype = tf.float16)

In [ ]:
model.fit(X, y, batch_size=64, epochs=5, validation_split=0.2)

In [ ]:
preds = model.predict(X)

In [ ]:
plt.plot(y[:,0], color = 'green')
plt.plot(preds[:,0], color = 'red')
plt.show()
plt.plot(y[:,0] - preds[:,0], color = 'yellow')
plt.show()